In [ ]:
%pip install comet_ml # libreria a instalar

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.4/586.4 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 41.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.8/252.8 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.2/512.2 kB 37.1 MB/s eta 0:00:00
  Attempting uninstall: websocket-client
    Found existing installation: websocket-client 1.6.4
    Uninstalling websocket-client-1.6.4:
      Successfully uninstalled websocket-client-1.6.4
  Attempting uninstall: python-box
    Found existing installation: python-box 7.1.1
    Uninstalling python-box-7.1.1:
      Successfully uninstalled python-box-7.1.1


In [ ]:
%pip install dvc # Libreria para cargar DVC

In [ ]:
!dvc get-url https://dagshub.com/jorgevc/Gatos-y-Perros/raw/24eaabd8ca81ed89f780138c1e3955ccd1f3d155/data/DogsCats.zip # directorio

!
  0%|          |https://dagshub.com/jorgevc/Gatos-y0.00/? [00:00<?,        ?B/s]
  0%|          |https://dagshub.com/jorgevc/Gatos-y0.00/? [00:00<?,        ?B/s]
  0%|          |https://dagshub.com/jorgevc/Gat319k/? [00:00<00:00,    3.25MB/s]
  0%|          |https://dagshub.com/jorgevc/Ga1.70M/? [00:00<00:00,    9.54MB/s]
  0%|          |https://dagshub.com/jorgevc/Ga4.42M/? [00:00<00:00,    17.8MB/s]
  0%|          |https://dagshub.com/jorgevc/Ga7.15M/? [00:00<00:00,    20.1MB/s]
  0%|          |https://dagshub.com/jorgevc/Ga9.36M/? [00:00<00:00,    20.4MB/s]
  0%|          |https://dagshub.com/jorgevc/Ga14.8M/? [00:00<00:00,    32.0MB/s]
  0%|          |https://dagshub.com/jorgevc/Ga21.0M/? [00:00<00:00,    42.2MB/s]
  0%|          |https://dagshub.com/jorgevc/Ga29.5M/? [00:00<00:00,    56.0MB/s]
  0%|          |https://dagshub.com/jorgevc/Ga33.4M/? [00:00<00:00,    51.5MB/s]
  0%|          |https://dagshub.com/jorgevc/Ga37.1M/? [00:01<00:00,    47.4MB/s]
  0%|          |https://da

In [ ]:
import zipfile # Zipfile ayuda en descomprimir las carpetas zip
with zipfile.ZipFile('DogsCats.zip','r') as zip_ref:
  zip_ref.extractall('/content')

In [ ]:
!ls # Para ver los archivos del directorio en el que se está

DogsCats.zip  sample_data  test  train


In [ ]:
# Imports
import comet_ml
comet_ml.init(project_name="perrosgatos")

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D, Dropout,Activation,MaxPooling2D,Flatten,SpatialDropout2D, Softmax
from tensorflow.keras.optimizers import RMSprop, SGD, Adam
import tensorflow.keras.regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from keras.callbacks import ModelCheckpoint


Please paste your Comet API key from https://www.comet.com/api/my/settings/
(api key may not show as you type)
Comet API key: ··········


COMET INFO: Valid Comet API Key saved in /root/.comet.config (set COMET_CONFIG to change where it is saved).


In [ ]:
# Graficas en comet
experiment = comet_ml.Experiment(
    auto_histogram_weight_logging=True,
    auto_histogram_gradient_logging=True,
    auto_histogram_activation_logging=True,
    log_code=True,

)
experiment.set_name('Experimento_4.6 todos los datos')

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/nicglowss/perrosgatos/562791f537d149af81ed2d350a0de692
COMET INFO:   Others:
COMET INFO:     Name : Experimento_4.6 todos los datos
COMET INFO:   Uploads:
COMET INFO:     environment details : 1
COMET INFO:     filename            : 1
COMET INFO:     installed packages  : 1
COMET INFO:     notebook            : 2
COMET INFO:     os packages         : 1
COMET INFO:     source_code         : 1
COMET INFO: 
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Couldn't find a Git repository in '/content

In [ ]:
# Pre-Procesamiento

ih, iw = 150, 150 #tamano de la imagen
input_shape = (ih, iw, 3) # alto, largo, rgb

train_dir = 'train' #directorio de entrenamiento
test_dir = 'test' #directorio de prueba

num_class = 2 # no. de clases
epochs = 50 # no. de epocas
            # por la tarea pasada despues de 50 me da sobreajuste

batch_size = 200 # Bajamos el no del batch por tiempo al 50%
num_train = 20000 #numero de imagenes en train
num_test = 5000 #numero de imagenes en test


epoch_steps = num_train // batch_size
test_steps = num_test // batch_size


gentrain = ImageDataGenerator(rescale=1. / 255., # normalizacion de las imagenes y el escalamiento
                              rotation_range=40,
                              width_shift_range=0.2,
                              height_shift_range=0.2,
                              shear_range=0.2,
                              zoom_range=0.2,
                              horizontal_flip=True,
                              fill_mode='nearest')
                                                  # Parametros que trabajan con la imagen de forma que se vea
                                                  # de distintos puntos y formas

train = gentrain.flow_from_directory(train_dir,
                batch_size=batch_size,
                target_size=(iw, ih),
                class_mode='binary')

gentest = ImageDataGenerator(rescale=1. / 255)

test = gentest.flow_from_directory(test_dir,
                batch_size=batch_size,
                target_size=(iw, ih),
                class_mode='binary')

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [ ]:
# Parametros

#para cargar pesos de la red desde donde se quedó la ultima vez
filename = "cvsd.h5"
model.load_weights(filename)  #comentar si se comienza desde cero.
                              # no funcionó, pidió definir model.
### Parametros
parameters = {
    "batch_size": batch_size,
    "epochs": epochs,
    "optimizer": 'SGD',
    "loss": 'binary_crossentropy',
}
# Arquitectura
experiment.log_parameters(parameters)

model = Sequential()
model.add(Conv2D(128, (3, 3), input_shape=(ih, iw,3),)) #el tamaño del recorte en el cuadrante
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2))) # pool-size es el tamaño de cuadrante

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dropout(0.2)) # Parametro por conveniencia
model.add(Flatten())
model.add(Dense(256)) # no. de neuronas
model.add(Activation('relu'))
model.add(Dense(1))
model.add(Activation('sigmoid'))


model.summary()

filepath = "m_pg_1.hdf5" # donde se guarda el progreso
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

model.compile(loss=parameters['loss'],
              optimizer=parameters['optimizer'],
              metrics=['accuracy'])

# Metodo de la clase model, model es la arquitectura y el .fit es para el entrenamiento

historial= model.fit(
              train,
              steps_per_epoch=epoch_steps,
              epochs=epochs,
              validation_data=test,
              validation_steps=test_steps,
              verbose=1,
              callbacks=[checkpoint])

experiment.log_model("Perros_y_Gatos", "m_pg_1.hdf5")
experiment.end()
model.save('cvsd.h5')

NameError: ignored